In [1]:
#!pip install numpy
#!pip install -q mediapipe
#!pip install wget
#!pip install opencv-python

In [2]:
import numpy as np
import cv2 as cv
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

In [3]:
import wget
wget.download('https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task')

'hand_landmarker (1).task'

即時手部偵測

In [7]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 指定模型路徑
model_path = r"C:\Users\User\yolo_yo\predict\hand_landmarker.task"
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

cap = cv2.VideoCapture(0)

# 設定新影片尺寸
new_width = 750
new_height = 480

def draw_landmarks_on_image(image, detection_result, thickness=3):
    """在影像上繪製手部關鍵點與連線"""
    annotated_image = image.copy()
    
    if detection_result.hand_landmarks:
        for landmarks in detection_result.hand_landmarks:
            # 繪製關鍵點
            for landmark in landmarks:
                x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                cv2.circle(annotated_image, (x, y), radius=7, color=(255, 0, 0), thickness=-1)  # 紅色點

            # 繪製關鍵點間的線條
            connections = mp.solutions.hands.HAND_CONNECTIONS
            for connection in connections:
                start_idx, end_idx = connection
                start_point = (int(landmarks[start_idx].x * image.shape[1]), int(landmarks[start_idx].y * image.shape[0]))
                end_point = (int(landmarks[end_idx].x * image.shape[1]), int(landmarks[end_idx].y * image.shape[0]))
                cv2.line(annotated_image, start_point, end_point, (0, 255, 0), thickness)  # 綠色線條
    return annotated_image


# 讀取影片並進行手部偵測
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # 影片結束則跳出迴圈

    # 調整影片大小
    frame = cv2.resize(frame, (new_width, new_height))

    # 轉換 OpenCV 的 BGR 影像為 RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # 偵測手部關鍵點
    detection_result = detector.detect(mp_image)

    # 繪製關鍵點
    annotated_frame = draw_landmarks_on_image(frame, detection_result)

    # 顯示影像
    cv2.imshow("Hand Landmark Detection", annotated_frame)

    if cv2.waitKey(30) & 0xFF == ord(' '):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()